# Visium preprocessing – load all 14 slides, clean spots, and export one H5AD  
In this notebook we read every 10x Visium slide from *Vanrobaeys et al.* (2023),  
attach pixel coordinates & subregion labels, drop off‑tissue spots and create a single
`adata_vis` object.  At the end we perform a few sanity plots to make sure counts
and coordinates look sensible.


In [2]:
# ─────────── Dask config patch ───────────
import dask
# Turn on the new “expression” engine so spatialdata can import
dask.config.set({"dataframe.query-planning": True})

# Now import the rest:
import squidpy as sq
import scanpy as sc
import pandas as pd
import numpy as np
import os, glob, re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# ───────────────────────────────────────────


NotImplementedError: The legacy implementation is no longer supported

## 1 · Import spatial toolkits and detect slide folders  
The first code block brings in *Squidpy* (spatial analysis) and *Scanpy* (AnnData
backend), then lists every directory that matches `Sample*`.  
We expect to see **14 folders**; if not, double‑check the path.


In [1]:
# Code 1 – Imports & folder scan
import squidpy as sq, scanpy as sc, pandas as pd, numpy as np, os, glob, re, matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline

ROOT = "../data"               # change if your slides live elsewhere
slide_dirs = sorted(glob.glob(os.path.join(ROOT, "Sample*")))

print(f"Found {len(slide_dirs)} candidate folders:")
for d in slide_dirs:
    print("  •", os.path.basename(d))
assert len(slide_dirs) == 14, "⚠️ Expected 14 slides – please verify the path!"


c:\Users\viraj\anaconda3\envs\stat530\lib\site-packages\anndata\utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
c:\Users\viraj\anaconda3\envs\stat530\lib\site-packages\anndata\utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
c:\Users\viraj\anaconda3\envs\stat530\lib\site-packages\anndata\utils.py:434: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
c:\Users\viraj\anaconda3\envs\stat530\lib\site-packages\anndata\utils.py:434: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
c:\Users\viraj\anaconda3\envs\stat530\lib\site-packages\anndata\utils.py:434: FutureWarning: Importing read_mtx from `anndata` is deprecat

NotImplementedError: The legacy implementation is no longer supported

## 2 · Load each slide, join `tissue_positions_list.csv`, and remove off‑tissue spots  
For every slide we:  
1.  Load the filtered HDF5 counts.  
2.  Merge pixel coordinates & `in_tissue` flag.  
3.  Attach any available `subregion_map.csv`.  
4.  Drop spots where `in_tissue == 0`.  
We print the number of remaining tissue spots so we can spot obvious mistakes
(very low counts would signal a mis‑matched barcode list).


In [ ]:
# Code 2 – Iterate and build a list of single‑slide AnnData objects
vis_list = []

for sdir in slide_dirs:
    sid   = os.path.basename(sdir)             # e.g. "Sample1SOR"
    h5    = os.path.join(sdir, "filtered_feature_bc_matrix.h5")
    tpos  = os.path.join(sdir, "tissue_positions_list.csv")
    subr  = os.path.join(sdir, "subregion_map.csv")
    
    # 1) Load raw counts + feature table
    ad    = sq.datasets.visium_10x_h5(h5)
    ad.var_names_make_unique()
    ad.obs_names_make_unique()
    ad.obs["sample"] = sid
    
    # 2) Merge pixel coordinates & tissue flag
    tdf = pd.read_csv(tpos, header=None,
                      names=["barcode","in_tissue","array_row","array_col","pxl_row","pxl_col"]).set_index("barcode")
    ad.obs = ad.obs.join(tdf.loc[ad.obs_names])
    
    # 3) Optional: subregion labels
    if os.path.exists(subr):
        srf = pd.read_csv(subr).set_index("barcode")
        ad.obs = ad.obs.join(srf, how="left")
    
    # 4) Keep in‑tissue spots only
    pre = ad.n_obs
    ad  = ad[ad.obs["in_tissue"] == 1].copy()
    print(f"✓ {sid}: kept {ad.n_obs}/{pre} on‑tissue spots")
    
    vis_list.append(ad)


## 3 · Concatenate all slides to one `adata_vis` object  
We glue the list together, attaching the `sample` label as a batch key.
Then we plot the spot count per slide to catch imbalances (e.g., slides with
almost no spots left).


In [ ]:
# Code 3 – Merge & bar chart
adata_vis = vis_list[0].concatenate(*vis_list[1:], join="outer",
                                    batch_key="sample",
                                    batch_categories=[ad.obs["sample"][0] for ad in vis_list])

print("Merged AnnData shape (spots × genes):", adata_vis.shape)
print("Unique samples:", adata_vis.obs["sample"].unique())

# Bar plot – spots per sample
spot_counts = adata_vis.obs["sample"].value_counts().sort_index()
spot_counts.plot(kind="barh", figsize=(6,4), color="steelblue")
plt.title("Number of tissue spots per slide")
for i, v in enumerate(spot_counts.values):
    plt.text(v+10, i, str(v), va="center")
plt.show()


## 4 · Build spatial neighbour graphs (6 nearest)  
Squidpy needs a neighbour graph to run Moran’s I, Hotspot, etc.  
We use the Visium grid coordinates (`array_row/array_col`) so distance reflects
physical neighbourhood on the slide.


In [ ]:
# Code 4 – Spatial neighbours
sq.gr.spatial_neighbors(adata_vis, coord_type="grid", n_neigh=6)
print("✓ Spatial neighbour graph stored in `adata_vis.obsp['spatial_connectivities']`")
print("Non‑zero connections:", adata_vis.obsp["spatial_connectivities"].nnz)


## 5 · Normalise counts per spot and log‑transform  
This quick scaling makes gene expression roughly comparable between spots.
We will redo a more sophisticated normalisation if needed later, but log‑CP10K
is fine for initial IEG hotspot detection.


In [ ]:
# Code 5 – Normalise
sc.pp.normalize_total(adata_vis, target_sum=1e4)
sc.pp.log1p(adata_vis)
print("Log‑CP10K normalisation complete.")


## 6 · Visual sanity check – plot Fos counts on a sample slide  
We pick a hippocampal slide (e.g., `Sample1SOR`) and see if *Fos* has any
visible hotspots.  A blank plot or uniformly zero spots would indicate a
gene‑name mismatch or failed normalisation.


In [ ]:
# Code 6 – Quick spatial plot
target = "Sample1SOR"
test   = adata_vis[adata_vis.obs["sample"] == target].copy()

if "Fos" in test.var_names:
    sq.pl.spatial_scatter(test, color="Fos", cmap="inferno", figsize=(4,4))
    plt.title(f"Fos counts – {target}")
else:
    print("⚠️ Fos not found; check gene‑name casing or annotation file.")


## 7 · Save the concatenated, normalised Visium dataset  
The file `visium_all.h5ad` will be read by Notebook 03 for IEG hotspot analysis.


In [ ]:
# Code 7 – Write file
adata_vis.write_h5ad("visium_all.h5ad", compression="gzip")
print("🎉  Saved to visium_all.h5ad  – ready for downstream IEG mapping")
